In [35]:
import pandas as pd
import numpy as np
import pathlib as pl
# library used for initial visualization
# library used for initial visualization
import sklearn
import pathlib as pl

In [36]:
path = 'C:/Users/Rolando/Downloads/financialphrases/all-data.csv'

df = pd.read_csv(path, encoding = "ISO-8859-1", names = ['sentiment', 'sentence'])

In [37]:
df

,sentiment,sentence
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [38]:
df['sentence']

0       According to Gran , the company has no plans t...
1       Technopolis plans to develop in stages an area...
2       The international electronic industry company ...
3       With the new production plant the company woul...
4       According to the company 's updated strategy f...
                              ...                        
4841    LONDON MarketWatch -- Share prices ended lower...
4842    Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843    Operating profit fell to EUR 35.4 mn from EUR ...
4844    Net sales of the Paper segment decreased to EU...
4845    Sales in Finland decreased by 10.5 % in Januar...
Name: sentence, Length: 4846, dtype: object

In [39]:
# path = 'C:/Users/Rolando/Downloads/newsTitles/SEntFiN-v1.1.csv'

# df = pd.read_csv(path, encoding = "ISO-8859-1")

In [40]:
# df

In [41]:
# path = 'C:/Users/Rolando/Downloads/newsTitles2/data.csv'

# df = pd.read_csv(path, encoding = "ISO-8859-1")

In [42]:
# df

In [43]:
# !pip install -U spacy

In [71]:
text = df.sample(1)['sentence'].iloc[0]

SPACY

In [72]:
import spacy

nlp = spacy.load("en_core_web_md")
doc = nlp(text)
print(doc.ents)

(some EUR 5 million, USD 6.5 m, annual)


RAKE NLTK

In [73]:
from rake_nltk import Rake
rake_nltk_var = Rake()

rake_nltk_var.extract_keywords_from_text(text)
keyword_extracted = rake_nltk_var.get_ranked_phrases()
print(keyword_extracted)

['eur 5 million', 'usd 6', 'cost savings', 'annual basis', '5', 'plan', 'generate', 'estimated']


TFIDF

In [74]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as sw

def train_keyword(data): #SE PUEDE EXTENDER EL DICCIONARIO
#     # use stopword to filter words    
#     stop = sw.stopwords_key()
#     stop = stop.splitlines()
#     stop = str(stop)
#     stop = stop.translate(str.maketrans('', '', string.punctuation))
    stopwords_list = list(sw.words('english'))
#     stopwords_list.extend(stop.split())
    
    # vectorizing the words use tf-idf
    vectorizer = TfidfVectorizer(analyzer='word', stop_words=set(stopwords_list), use_idf=True)
    vector = vectorizer.fit_transform(data)
    
    # store to dataframe    
    df = pd.DataFrame(vector[0].T.todense(), index=vectorizer.get_feature_names(), columns=["vector"])
    df = df.sort_values('vector', ascending=False)
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'keywords'})
    df = df[df['vector'] != 0]

    return df

In [75]:
raw_data = [text]
keywords = train_keyword(raw_data)
# print top 10 keywords
print(keywords.head(10))

    keywords    vector
0     annual  0.316228
1      basis  0.316228
2       cost  0.316228
3  estimated  0.316228
4        eur  0.316228
5   generate  0.316228
6    million  0.316228
7       plan  0.316228
8    savings  0.316228
9        usd  0.316228


KEYBERT

In [76]:
# # Install the libraries
# !pip install keybert
# !pip install sentence-transformers

In [77]:
# Import them
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

# To show the execution times
# !pip install ipython-autotime
# %load_ext autotime

In [78]:
sent_trans = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
keyBERT_model = KeyBERT(model = sent_trans)

In [85]:
def extract_terms(document, n_gram_range = (3,3), 
                  top_N=5, model=keyBERT_model, diversity_threshold = 0.7):

  keywords = model.extract_keywords(document, stop_words='english', 
                                    keyphrase_ngram_range=(1, 3),
                                    use_mmr=True, 
                                    diversity = diversity_threshold,
                                    top_n = top_N)
  
  return sorted(keywords, key=lambda tup:(-tup[1], tup[0]))

In [86]:
print(f'Text: {text}.\nLength: {len(text.split())}')
best_terms = extract_terms(text)

for terms_and_score in best_terms:
  print(terms_and_score)

Text: The plan is estimated to generate some EUR 5 million ( USD 6.5 m ) in cost savings on an annual basis ..
Length: 23
('cost savings annual', 0.5425)
('generate eur million', 0.3755)
('plan', 0.3277)
('usd cost', 0.2052)
('basis', 0.0321)
